<h1>Portfolio Monitor</h1>
<h2>Updated portfolio and stock market evaluation</h2>




<h6>
    <p>VERSION: 1.0</p>
    <p>Stock-Monitor Update from Swissquote Transaction file</p>
    <p>Roberto Schlatter</p>
    <p>19-07-2021</p>
</h6>

<h3>
Prerequisite
</h3>

In [2]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
import datetime
import pickle
import os.path
import os
import csv
#from googleapiclient.discovery import build
#from google_auth_oauthlib.flow import InstalledAppFlow
#from google.auth.transport.requests import Request
#from google.protobuf import service


<h3>
Upload to Google Sheet Function
</h3>

In [3]:
def storeDataToGoogleSheet(SAMPLE_SPREADSHEET_ID_input, SAMPLE_RANGE_NAME, df_gold, sOrientation, bSingleValue):
    def Create_Service(client_secret_file, api_service_name, api_version, *scopes):
        global service
        SCOPES = [scope for scope in scopes[0]]
        #print(SCOPES)
        cred = None

        if os.path.exists('token_write.pickle'):
            with open('token_write.pickle', 'rb') as token:
                cred = pickle.load(token)

        if not cred or not cred.valid:
            if cred and cred.expired and cred.refresh_token:
                cred.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(client_secret_file, SCOPES)
                cred = flow.run_local_server()

            with open('token_write.pickle', 'wb') as token:
                pickle.dump(cred, token)

        try:
            service = build(api_service_name, api_version, credentials=cred)
            print(api_service_name, 'service created successfully')
            #return service
        except Exception as e:
            print(e)
            #return None

    # change 'my_json_file.json' by your downloaded JSON file: my_credentials.json
    Create_Service('my_credentials.json', 'sheets', 'v4',['https://www.googleapis.com/auth/spreadsheets'])

    if(bSingleValue):
        def SingleValue_Export_Data_To_Sheets():
            response_date = service.spreadsheets().values().update(
                spreadsheetId=SAMPLE_SPREADSHEET_ID_input,
                valueInputOption='RAW',
                range=SAMPLE_RANGE_NAME,
                body=dict(
                    #Orientation: ROWS oder COLUMNS
                    majorDimension=sOrientation,
                    values=df_gold.T.reset_index().T.values.tolist())
            ).execute()
            print('Sheet successfully Updated')
        SingleValue_Export_Data_To_Sheets()
    else:
        def Export_Data_To_Sheets():
            response_date = service.spreadsheets().values().update(
                spreadsheetId=SAMPLE_SPREADSHEET_ID_input,
                valueInputOption='RAW',
                range=SAMPLE_RANGE_NAME,
                body=dict(
                    #Orientation: ROWS oder COLUMNS
                    majorDimension=sOrientation,
                    values=df_gold.T.reset_index().T.values.tolist())
            ).execute()
            print('Sheet successfully Updated')
        Export_Data_To_Sheets()

  <h3>Pre Definition for Test run and final run</h3>

In [4]:
sTestRun = False

if sTestRun:
    sGoogleSheet = "17iBlVZySEFsqI9RjPR6hVrlD_QtQ-zc46bKuar0nFOk"
else:
    sGoogleSheet = "1RX2lJwZYWUO_sJUME1M9CsMFkGQNEqMCiY2Y6vCZjF0"

<h3>Swissquote Transaction import</h3>

In [5]:
sFilename = 'transactions-from-01012010-to-14072021.xlsx'

# Set new header names in order to remove Umlaute
lst_column_names = ['Auftrag', 'Datum', 'Transaktionen', 'Symbol', 'Name', 'ISIN', 'Anzahl', 'Stueckpreis', 'Kosten','Aufgelaufene Zinsen','Nettobetrag','Waehrung Nettobetrag', 'Nettobetrag in der Waehrung des Kontos','Saldo','Waehrung']

# Dataframe initialisieren!
df_transactions = pd.DataFrame(columns = lst_column_names)

df_transactions = df_transactions.append(pd.read_excel(sFilename, index_col=1, names=lst_column_names))

# Dataframe at the end of the cell without print shows the formatted table
df_transactions.head(10)

,Auftrag,Datum,Transaktionen,Symbol,Name,ISIN,Anzahl,Stueckpreis,Kosten,Aufgelaufene Zinsen,Nettobetrag,Waehrung Nettobetrag,Nettobetrag in der Waehrung des Kontos,Saldo,Waehrung
0,2021-07-09 11:03:34,NaN,Buy,ADA,ADA,EX0X00ADA000,150.0,1.32,1.98,0,-199.98,USD,-199.98,27.55,USD
0,2021-07-05 11:22:08,NaN,Dividende,VWRL,Vanguard All World ETF Dist,IE00B3RBWM25,1.0,113.92,0.00,0,113.92,USD,113.92,227.53,USD
0,2021-07-01 13:07:48,NaN,Dividende,VFEM,Vanguard EM ETF Dist,IE00B3VVMM84,1.0,134.61,0.00,0,134.61,USD,134.61,113.61,USD
0,2021-06-30 22:19:51,NaN,Fx-Gutschrift Comp.,NaN,NaN,NaN,1.0,141.34,0.00,0,141.34,USD,141.34,-21.00,USD
0,2021-06-30 22:19:51,NaN,Fx-Belastung Comp.,NaN,NaN,NaN,1.0,132.00,0.00,0,-132,CHF,-132,0.00,CHF
0,2021-06-30 20:59:16,NaN,Depotgebühren,NaN,NaN,NaN,1.0,48.00,3.70,0,-51.7,CHF,-51.7,132.00,CHF
88666265,2021-06-30 19:27:07,NaN,Kauf,NIO,NIO ADS REP 1 CL A ORD,US62914V1061,6.0,53.19,10.33,0,-329.47,USD,-329.47,-162.34,USD
0,2021-06-30 17:42:12,NaN,Dividende,VJPN,Vanguard Japan ETF Dist,IE00B95PGT31,1.0,69.43,0.00,0,69.43,USD,69.43,167.13,USD
0,2021-06-29 17:00:42,NaN,Dividende,VTI,VANGUARD TOTAL STOCK MARKET ET,US9229087690,1.0,118.18,35.46,0,82.72,USD,82.72,97.70,USD
88397267,2021-06-24 09:01:05,NaN,Kauf,VJPN,Vanguard Japan ETF Dist,IE00B95PGT31,21.0,32.89,1.05,0,-691.75,CHF,-691.75,183.70,CHF


<h5>Dividends</h5>

In [6]:
lst_div_names = ['Jahr', 'Monat', 'Symbol', 'Nettobetrag in der Waehrung des Kontos']
df_dividende = pd.DataFrame(columns = lst_div_names)
df_dividende = df_dividende.append(df_transactions[df_transactions['Transaktionen'].isin(['Dividende'])])


# Nach Datum sortiert - für Monatsübersicht
#Create new Columns with Jahr and Monat as extracted from "Datum"
df_dividende['Jahr'] = pd.DatetimeIndex(df_dividende['Auftrag']).year
df_dividende['Monat'] = pd.DatetimeIndex(df_dividende['Auftrag']).month


lst_header = ['Jahr', 'Monat', 'Symbol']

df_dividende_year_month = df_dividende.groupby(lst_header)['Nettobetrag in der Waehrung des Kontos'].sum()                                                                                                   
                                                                                                     
#-----------------------------------------
# Call Upload: Dividend detail table by month and year
#-----------------------------------------
#storeDataToGoogleSheet(sGoogleSheet, 'HISTORIC-DATA!M03', df_dividende_year_month, 'COLUMNS', False)

df_dividende_year_month

Jahr  Monat  Symbol
2012  3      ROG        22.10
      8      AAPL        9.27
      11     AAPL        9.27
2015  7      SCE        49.77
2016  1      SPY5       21.36
                        ...  
2021  6      MSFT       28.62
             VJPN       69.43
             VTI        82.72
      7      VFEM      134.61
             VWRL      113.92
Name: Nettobetrag in der Waehrung des Kontos, Length: 94, dtype: float64

In [7]:
# df_dividende['Nettobetrag in der Währung des Kontos'] = pd.to_numeric(df_dividende['Nettobetrag in der Waehrung des Kontos'])

lst_header = ['Symbol']

df_export_dividende_summary = df_dividende.groupby(lst_header)['Nettobetrag in der Waehrung des Kontos'].sum().head(25)

#-----------------------------------------
# Call Upload: Dividend summary on first page
#-----------------------------------------
#storeDataToGoogleSheet(sGoogleSheet,'SUMMARY!B56', df_export_dividende_summary, 'COLUMNS', False)

df_export_dividende_summary

Symbol
AAPL       168.17
AMS         38.08
AXP         27.39
BLV         30.88
FCAU      3805.53
IBTM        24.72
MSFT       116.22
NVDA        12.82
PGHN       220.35
QCOM       101.66
ROG       1615.89
SCE         49.77
SMMCHA     316.92
SPY5        43.50
STLA       470.85
TLT        152.56
TSM        141.25
VFEM       449.45
VJPN        69.43
VTI        501.07
VWRL       401.46
WM          27.86
XDJP       140.08
Name: Nettobetrag in der Waehrung des Kontos, dtype: float64

<h5>Costs</h5>

In [8]:
df_Kosten_total = pd.DataFrame()
df_Kosten_total = df_Kosten_total.append(df_transactions[df_transactions['Transaktionen'].isin(['Kauf', 'Verkauf'])])

df_Kosten_total['Jahr'] = pd.DatetimeIndex(df_Kosten_total['Auftrag']).year
df_Kosten_total['Monat'] = pd.DatetimeIndex(df_Kosten_total['Auftrag']).month

# Final dataframe for costs
lst_header_cost = ['Transaktionen','Symbol', 'Jahr', 'Monat']
df_export_Kosten_year = df_Kosten_total.groupby(lst_header_cost)['Kosten'].sum()

#-----------------------------------------
# Call Upload: Cost per asset and month & year
#-----------------------------------------
#storeDataToGoogleSheet(sGoogleSheet, 'HISTORIC-DATA!A03', df_export_Kosten_year, 'COLUMNS',False)

df_export_Kosten_year

Transaktionen  Symbol  Jahr  Monat
Kauf           AAPL    2012  3        40.34
                       2016  2        42.43
                       2018  10       35.88
                             11       28.37
                       2019  1        27.59
                                      ...  
Verkauf        VTI     2020  3        14.93
               WM      2020  9         8.37
               XDJP    2020  9        72.02
               XMCH    2020  4        46.85
               ZM      2020  4         8.39
Name: Kosten, Length: 224, dtype: float64

In [9]:
lst_header_cost = ['Symbol']
df_export_Kosten_total = df_Kosten_total.groupby(lst_header_cost)['Kosten'].sum()

#-----------------------------------------
# Call Upload: Cost per asset in total
#-----------------------------------------
#storeDataToGoogleSheet(sGoogleSheet, 'HISTORIC-DATA!G03', df_export_Kosten_total, 'COLUMNS',False)

df_export_Kosten_total

Symbol
AAPL    464.58
ABNB     68.94
ACIC     75.97
ACIU     42.40
ACWI     71.65
         ...  
VWRL    114.85
WM       87.66
XDJP    217.97
XMCH    139.50
ZM       47.12
Name: Kosten, Length: 66, dtype: float64

<h5>Total Performance</h5>

<h3>Historic Performance if buy and hold up to today</h3>

In [10]:
df_Performance_alltime = pd.DataFrame()
df_Performance_qty = pd.DataFrame()

df_Performance_alltime = df_Performance_alltime.append(df_transactions[df_transactions['Transaktionen'].isin(['Kauf'])])

lst_header_perf = ['Symbol']

df_Performance_qty = df_Performance_qty.append(df_transactions[df_transactions['Transaktionen'].isin(['Kauf'])])
df_Performance_qty = df_Performance_qty.groupby(lst_header_perf)['Anzahl'].sum()


# Remove tausender Trennzeichen um Kolonne als Zahlenwert beim Summieren zu verewenden
df_Performance_alltime = df_Performance_alltime.groupby(lst_header_perf)['Stueckpreis'].mean()
df_Performance_alltime

df_performance_alltime_all = pd.DataFrame(data=df_Performance_alltime)

df_performance_alltime_all['Gewinn/Verlust'] = df_Performance_assets
df_performance_alltime_all['QTY'] = df_Performance_qty
df_performance_alltime_all

NameError: name 'df_Performance_assets' is not defined

<h5>Buy, Sell and Withdrawl Balance - including Crypto</h5>

In [13]:
# get current date and format it
pUpdateDate = datetime.datetime.now()
pDay = pUpdateDate.strftime("%A")
pDate = pUpdateDate.strftime("%d-%m-%Y | %H:%M:%S")
lst_date = [pDay, pDate]

# number of current assets - KAUF / VERKAUF
df_numberAssets_k = pd.DataFrame()
df_numberAssets_count_k = pd.DataFrame()

df_numberAssets_k = df_numberAssets_k.append(df_transactions[df_transactions['Transaktionen'].isin(['Kauf'])])
#df_numberAssets_k = df_numberAssets_k[['Symbol','Anzahl']]
df_numberAssets_count_k = df_numberAssets_k.groupby('Symbol')['Anzahl'].sum()

# Portfolio: Anzahl Positionen KAUF
df_numberAssets_count_k


Symbol
AAPL    150.0
ABNB     14.0
ACIC    500.0
ACIU     38.0
ACWI      8.0
        ...  
VWRL    196.0
WM       44.0
XDJP    547.0
XMCH    500.0
ZM       35.0
Name: Anzahl, Length: 63, dtype: float64

In [14]:
df_numberAssets_v = pd.DataFrame()
df_numberAssets_count_v = pd.DataFrame()
df_current_Portfolio = pd.DataFrame()
df_quantity_adjustments = pd.DataFrame({'Symbols': ['FCAU'],'Anzahl': ['2000']})

df_numberAssets_v = df_transactions[df_transactions['Transaktionen'].isin(['Verkauf'])]
df_numberAssets_v = df_numberAssets_v[['Symbol','Anzahl']]
df_numberAssets_count_v = df_numberAssets_v.groupby('Symbol')['Anzahl'].sum()

# Aktuelles Portfolio - Fill value = 0
df_current_Portfolio = df_numberAssets_count_k.sub(df_numberAssets_count_v,fill_value=0)

df_current_Portfolio[df_current_Portfolio >0]

#-----------------------------------------
# Call Upload: Current Portfolio
#-----------------------------------------
#storeDataToGoogleSheet(sGoogleSheet, 'HISTORIC-DATA!BU03', df_current_Portfolio, 'COLUMNS', False)


Symbol
AUCHAH     124.0
BYND        33.0
FB          60.0
FCAU      2000.0
ILMN        20.0
NIO         72.0
PGHN         8.0
PLTR       220.0
RBLX        80.0
SMMCHA     127.0
VFEM       335.0
VJPN       320.0
VTI        175.0
VWRL       196.0
Name: Anzahl, dtype: float64

<h3>Total Performance</h3>

In [37]:
df_Performance_assets = pd.DataFrame()
df_Performance_buy = pd.DataFrame()
df_Performance_sell = pd.DataFrame()

df_Performance_buy = df_Performance_buy.append(df_transactions[df_transactions['Transaktionen'].isin(['Kauf'])])

lst_header_perf = ['Symbol']

# Remove tausender Trennzeichen um Kolonne als Zahlenwert beim Summieren zu verewenden
df_Performance_buy = df_Performance_buy.groupby(lst_header_perf)['Nettobetrag in der Waehrung des Kontos'].sum()

df_Performance_sell = df_Performance_sell.append(df_transactions[df_transactions['Transaktionen'].isin(['Verkauf'])])

# Remove tausender Trennzeichen um Kolonne als Zahlenwert beim Summieren zu verewenden
df_Performance_sell['Nettobetrag in der Währung des Kontos'] = df_Performance_sell['Nettobetrag in der Waehrung des Kontos'].replace("'")

df_Performance_sell = df_Performance_sell.groupby(lst_header_perf)['Nettobetrag in der Waehrung des Kontos'].sum()

df_Performance_assets =  df_Performance_sell - (-df_Performance_buy)
print(df_Performance_assets)

col_1 = {"Symbol","Value"}
col_2 = {"Symbol","Anzahl"}

df_1 = pd.DataFrame(columns=col_1, data=df_Performance_assets)
df_2 = pd.DataFrame(columns=col_2, data=df_current_Portfolio)

print("1: ",df_1)
df_2.Anzahl >0
#df_Performance_assets = pd.merge(df_Performance_assets,df_current_Portfolio, on='Symbol', how='outer' )


#df_Performance_assets.dropna(inplace=True)
#-----------------------------------------
# Call Upload: Performance per asset
#-----------------------------------------
#storeDataToGoogleSheet(sGoogleSheet, 'HISTORIC-DATA!R03', df_Performance_assets, 'COLUMNS', False)

##********
## Noch zum korrigieren - nur Assets welche auch komplett verkauft wurden berücksichtigen - ansonsten sind aktuelle Positionen im Minus
##********



#df_Performance_assets

Symbol
AAPL    4179.35
ABNB    -254.44
ACIC     -80.97
ACIU    -130.56
ACWI     -82.45
         ...   
VWRL        NaN
WM       359.46
XDJP      92.43
XMCH     184.60
ZM      -338.67
Name: Nettobetrag in der Waehrung des Kontos, Length: 66, dtype: float64
1:  Empty DataFrame
Columns: [Value, Symbol]
Index: []


Symbol
AAPL    False
ABNB    False
ACIC    False
ACIU    False
ACWI    False
        ...  
VWRL     True
WM      False
XDJP    False
XMCH    False
ZM      False
Name: Anzahl, Length: 66, dtype: bool

<h3>CRYPTHO</H3>

In [103]:
# number of current assets - CRYPTHO BUY
df_numberAssets_c = pd.DataFrame()
df_numberAssets_count_buy = pd.DataFrame()
df_numberAssets_c = df_numberAssets_c.append(df_transactions[df_transactions['Transaktionen'].isin(['Buy'])])
df_numberAssets_c = df_numberAssets_c[['Symbol','Anzahl']]
df_numberAssets_count_buy = df_numberAssets_c.groupby('Symbol')['Anzahl'].sum()

print(df_numberAssets_count_buy)

# Summe 'Initial Price' noch ausrechnen
df_numberAssets_c_nettobetrag = df_transactions[df_transactions['Transaktionen'].isin(['Buy'])]
df_numberAssets_c_nettobetrag = df_numberAssets_c_nettobetrag[['Symbol','Anzahl','Nettobetrag']]

#  Convert 'Nettobetrag' Column to abs values
df_numberAssets_count_c_nettobetrag = df_numberAssets_c_nettobetrag.groupby('Symbol').Nettobetrag.apply(lambda c: c.abs().sum())
df_numberAssets_count_c_nettobetrag

df_initialPrice = pd.DataFrame()
df_initialPrice = df_numberAssets_count_c_nettobetrag.div(df_numberAssets_count_buy, fill_value=1)

print("QTY: ", df_numberAssets_count_buy)
#print("Nettobetrag: ", df_numberAssets_count_c_nettobetrag)
print("Calc: ", df_initialPrice)






Symbol
ADA     2750.000
DOGE     530.000
ETH        6.162
XBT        0.550
Name: Anzahl, dtype: float64
QTY:  Symbol
ADA     2750.000
DOGE     530.000
ETH        6.162
XBT        0.550
Name: Anzahl, dtype: float64
Calc:  Symbol
ADA         1.460775
DOGE        0.326226
ETH      2090.529049
XBT     12867.909091
dtype: float64


In [102]:
# Summe 'Nettobetrag' noch ausrechnen
df_numberAssets_c_nettobetrag = df_transactions[df_transactions['Transaktionen'].isin(['Buy'])]
df_numberAssets_c_nettobetrag = df_numberAssets_c_nettobetrag[['Symbol','Anzahl','Nettobetrag']]

#  Convert 'Nettobetrag' Column to abs values
df_numberAssets_count_c_nettobetrag = df_numberAssets_c_nettobetrag.groupby('Symbol').Nettobetrag.apply(lambda c: c.abs().sum())

#print("Nettobetrag Cryptho sum():", df_numberAssets_count_c_nettobetrag)

# number of current assets - CRYPTHO WITHDRAWAL
df_numberAssets_w = df_transactions[df_transactions['Transaktionen'].isin(['Withdrawal'])]
df_numberAssets_w = df_numberAssets_w[['Symbol','Anzahl']]
df_numberAssets_count_w = df_numberAssets_w.groupby('Symbol')['Anzahl'].sum()
df_numberAssets_count_w = pd.DataFrame(df_numberAssets_count_w)
df_numberAssets_count_w.fillna("0")
df_numberAssets_sell = df_transactions[df_transactions['Transaktionen'].isin(['Sell'])]
df_numberAssets_sell = df_numberAssets_sell[['Symbol','Anzahl']]
df_numberAssets_count_sell = df_numberAssets_sell.groupby('Symbol')['Anzahl'].sum()
df_numberAssets_count_sell = pd.DataFrame(df_numberAssets_count_sell)
df_numberAssets_count_sell.fillna("0")

#df_numberAssets_count_w = df_numberAssets_count_w.add(df_numberAssets_count_sell,fill_value=0) 

print("KAUF: ", df_numberAssets_count_buy)
print("VERKAUF: ", df_numberAssets_count_sell)
print("WITHDRAWL: ", df_numberAssets_count_w)

#-----------------------------------------
# Call Upload: CRYPTHO
#-----------------------------------------
# Anzahl Assets Kauf CRYPTHO
#storeDataToGoogleSheet(sGoogleSheet, 'HISTORIC-DATA!AD03', df_numberAssets_count_buy, 'COLUMNS', False)

# Anzahl Assets Verkauf CRYPTHO
#storeDataToGoogleSheet(sGoogleSheet, 'HISTORIC-DATA!AG03', df_numberAssets_count_sell, 'COLUMNS', False)

# Anzahl Initial Value Kauf CRYPTHO
#storeDataToGoogleSheet(sGoogleSheet, 'HISTORIC-DATA!BG03', df_initialPrice, 'COLUMNS', False)

# Current Portfolio
#storeDataToGoogleSheet(sGoogleSheet, 'HISTORIC-DATA!BC03', df_numberAssets_count_nettobetrag, 'COLUMNS', False)


KAUF:  Symbol
ADA     2750.000
DOGE     530.000
ETH        6.162
XBT        0.550
Name: Anzahl, dtype: float64
VERKAUF:              Anzahl
Symbol            
ADA     2000.00000
ETH        3.16200
XBT        0.38565
WITHDRAWL:           Anzahl
Symbol         
XBT     0.01435


<h5>Investment Balance</h5>

In [82]:
# EINZAHLUNGEN - 'INVESTMENT' pro Jahr
df_sumInvestment = df_transactions[df_transactions['Transaktionen'].isin(['Vergütung'])]
df_sumInvestment['Jahr'] = pd.DatetimeIndex(df_sumInvestment.iloc[:,0]).year
df_sumInvestment = df_sumInvestment[['Jahr', 'Nettobetrag']]
df_sumInvestment_total = df_sumInvestment.groupby('Jahr')['Nettobetrag'].sum()

# AUSZAHLUNGEN - 'AUSZAHLUNG' pro Jahr
df_sumInvestment_out = df_transactions[df_transactions['Transaktionen'].isin(['Auszahlung'])]
df_sumInvestment_out['Jahr'] = pd.DatetimeIndex(df_sumInvestment_out.iloc[:,0]).year
df_sumInvestment_out = df_sumInvestment_out[['Jahr', 'Nettobetrag']]
df_sumInvestment_out_total = df_sumInvestment_out.groupby('Jahr')['Nettobetrag'].sum()

df_sumInvestment_total_net=df_sumInvestment_total.sub(df_sumInvestment_out_total,fill_value=0)

#-----------------------------------------
# Call Upload: Investments per Year
#-----------------------------------------
#storeDataToGoogleSheet(sGoogleSheet, 'HISTORIC-DATA!BO03', df_sumInvestment_total_net, 'COLUMNS', False)
df_sumInvestment_total


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


Jahr
2011     1160.00
2012     4500.00
2014    18750.00
2015     6500.00
2016    10950.00
2017     7500.00
2018     8000.00
2019    33398.27
2020    60000.00
2021    20000.00
Name: Nettobetrag, dtype: float64

<h5>Google Sheets Update Function</h5>

In [12]:
def storeDataToGoogleSheet(SAMPLE_SPREADSHEET_ID_input, SAMPLE_RANGE_NAME, df_gold, sOrientation, bSingleValue):
    def Create_Service(client_secret_file, api_service_name, api_version, *scopes):
        global service
        SCOPES = [scope for scope in scopes[0]]
        #print(SCOPES)
        cred = None

        if os.path.exists('token_write.pickle'):
            with open('token_write.pickle', 'rb') as token:
                cred = pickle.load(token)

        if not cred or not cred.valid:
            if cred and cred.expired and cred.refresh_token:
                cred.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(client_secret_file, SCOPES)
                cred = flow.run_local_server()

            with open('token_write.pickle', 'wb') as token:
                pickle.dump(cred, token)

        try:
            service = build(api_service_name, api_version, credentials=cred)
            print(api_service_name, 'service created successfully')
            #return service
        except Exception as e:
            print(e)
            #return None

    # change 'my_json_file.json' by your downloaded JSON file: my_credentials.json
    Create_Service('my_credentials.json', 'sheets', 'v4',['https://www.googleapis.com/auth/spreadsheets'])

    if(bSingleValue):
        def SingleValue_Export_Data_To_Sheets():
            response_date = service.spreadsheets().values().update(
                spreadsheetId=SAMPLE_SPREADSHEET_ID_input,
                valueInputOption='RAW',
                range=SAMPLE_RANGE_NAME,
                body=dict(
                    #Orientation: ROWS oder COLUMNS
                    majorDimension=sOrientation,
                    values=df_gold.T.reset_index().T.values.tolist())
            ).execute()
            print('Sheet successfully Updated')
        SingleValue_Export_Data_To_Sheets()
    else:
        def Export_Data_To_Sheets():
            response_date = service.spreadsheets().values().update(
                spreadsheetId=SAMPLE_SPREADSHEET_ID_input,
                valueInputOption='RAW',
                range=SAMPLE_RANGE_NAME,
                body=dict(
                    #Orientation: ROWS oder COLUMNS
                    majorDimension=sOrientation,
                    values=df_gold.T.reset_index().T.values.tolist())
            ).execute()
            print('Sheet successfully Updated')
        Export_Data_To_Sheets()

<h5>Google Function Call</h5>

In [13]:


# Date & time stamp of update
#storeDataToGoogleSheet(sGoogleSheet, 'SUMMARY!C02', lst_date, 'ROWS', True)





# Performance calculation buy & sell price
storeDataToGoogleSheet(sGoogleSheet, 'HISTORIC-DATA!R03', df_export_performance_values, 'COLUMNS', False)


# Anzahl Assets Kauf
storeDataToGoogleSheet(sGoogleSheet, 'HISTORIC-DATA!X03', df_numberAssets_count_k, 'COLUMNS', False)

# Anzahl Assets Verkauf
storeDataToGoogleSheet(sGoogleSheet, 'HISTORIC-DATA!AA03', df_numberAssets_count_v, 'COLUMNS', False)





# Auszahlungen pro Jahr
storeDataToGoogleSheet(sGoogleSheet, 'HISTORIC-DATA!BR03', df_sumInvestment_out_total, 'COLUMNS', False)



sheets service created successfully


TypeError: unsupported operand type(s) for &: 'str' and 'str'

<h3>AlleAktien Freiheitsfonds Import</h3>

<h3>Chart Creation</h3>